<a href="https://colab.research.google.com/github/Saathik-Basha/DA-1/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install "ipython-sql>=0.5.0" "prettytable<3.10"
%load_ext sql


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.3 MB/s eta 0:00:00


In [2]:
import pandas as pd

url = "https://raw.githubusercontent.com/salemprakash/EDA/main/Data/CreditCard.csv"
df = pd.read_csv(url)

# Look at first 5 rows
df.head()


,rownames,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,1,yes,0,37.66667,4.5200,0.033270,124.983300,yes,no,3,54,1,12
1,2,yes,0,33.25000,2.4200,0.005217,9.854167,no,no,3,34,1,13
2,3,yes,0,33.66667,4.5000,0.004156,15.000000,yes,no,4,58,1,5
3,4,yes,0,30.50000,2.5400,0.065214,137.869200,no,no,0,25,1,7
4,5,yes,0,32.16667,9.7867,0.067051,546.503300,yes,no,2,64,1,5


In [4]:
import sqlite3

conn = sqlite3.connect("/content/project.db")

df.to_sql("creditcard", conn, if_exists="replace", index=False)

print("Dataset imported into SQLite as table 'creditcard'")



Dataset imported into SQLite as table 'creditcard'


In [5]:
%load_ext sql
%sql sqlite:////content/project.db
%config SqlMagic.style = 'PLAIN_COLUMNS'


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
%%sql
SELECT * FROM creditcard LIMIT 5;


 * sqlite:////content/project.db
Done.


rownames,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
1,yes,0,37.66667,4.52,0.03326991,124.9833,yes,no,3,54,1,12
2,yes,0,33.25,2.42,0.005216942,9.854167,no,no,3,34,1,13
3,yes,0,33.66667,4.5,0.004155556,15.0,yes,no,4,58,1,5
4,yes,0,30.5,2.54,0.06521378,137.8692,no,no,0,25,1,7
5,yes,0,32.16667,9.7867,0.06705059,546.5033,yes,no,2,64,1,5


In [10]:
%%sql
PRAGMA table_info(creditcard);




 * sqlite:////content/project.db
Done.


cid,name,type,notnull,dflt_value,pk
0,rownames,INTEGER,0,None,0
1,card,TEXT,0,None,0
2,reports,INTEGER,0,None,0
3,age,REAL,0,None,0
4,income,REAL,0,None,0
5,share,REAL,0,None,0
6,expenditure,REAL,0,None,0
7,owner,TEXT,0,None,0
8,selfemp,TEXT,0,None,0
9,dependents,INTEGER,0,None,0


In [12]:
%%sql

SELECT * FROM creditcard LIMIT 10;

SELECT COUNT(*) AS total_rows FROM creditcard;


 * sqlite:////content/project.db
Done.
Done.


total_rows
1319


In [15]:
%%sql
-- Count NULLs per column
SELECT
  SUM(CASE WHEN card IS NULL THEN 1 ELSE 0 END) AS null_card,
  SUM(CASE WHEN reports IS NULL THEN 1 ELSE 0 END) AS null_reports,
  SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS null_age,
  SUM(CASE WHEN income IS NULL THEN 1 ELSE 0 END) AS null_income,
  SUM(CASE WHEN share IS NULL THEN 1 ELSE 0 END) AS null_share,
  SUM(CASE WHEN expenditure IS NULL THEN 1 ELSE 0 END) AS null_expenditure,
  SUM(CASE WHEN owner IS NULL THEN 1 ELSE 0 END) AS null_owner,
  SUM(CASE WHEN selfemp IS NULL THEN 1 ELSE 0 END) AS null_selfemp,
  SUM(CASE WHEN dependents IS NULL THEN 1 ELSE 0 END) AS null_dependents,
  SUM(CASE WHEN months IS NULL THEN 1 ELSE 0 END) AS null_months,
  SUM(CASE WHEN majorcards IS NULL THEN 1 ELSE 0 END) AS null_majorcards,
  SUM(CASE WHEN active IS NULL THEN 1 ELSE 0 END) AS null_active
FROM creditcard;


 * sqlite:////content/project.db
Done.


null_card,null_reports,null_age,null_income,null_share,null_expenditure,null_owner,null_selfemp,null_dependents,null_months,null_majorcards,null_active
0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
%%sql

-- Create normalized income (0-1 scale)
SELECT
  income,
  (income - (SELECT MIN(income) FROM creditcard)) * 1.0 /
  ((SELECT MAX(income) FROM creditcard) - (SELECT MIN(income) FROM creditcard)) AS income_norm
FROM creditcard
LIMIT 10;


 * sqlite:////content/project.db
Done.


income,income_norm
4.52,0.32430398796087284
2.42,0.16629044394281414
4.5,0.3227990970654628
2.54,0.17531978931527467
9.7867,0.720594431903687
2.5,0.1723100075244545
3.96,0.28216704288939054
2.37,0.16252821670428896
3.8,0.2701279157261099
3.2,0.2249811888638074


In [19]:
%%sql
-- Convert REAL → INTEGER (round values)
UPDATE creditcard
SET age = CAST(age AS INTEGER);

-- Ensure months is integer
UPDATE creditcard
SET months = CAST(months AS INTEGER);


 * sqlite:////content/project.db
1319 rows affected.
1319 rows affected.


[]

In [20]:
%%sql
CREATE TABLE creditcard_dedup AS
SELECT *
FROM (
    SELECT *, ROW_NUMBER() OVER (
        PARTITION BY card, reports, age, income, share, expenditure, owner, selfemp, dependents, months, majorcards, active
        ORDER BY rowid
    ) AS rn
    FROM creditcard
)
WHERE rn = 1;


 * sqlite:////content/project.db
Done.


[]

In [25]:
%%sql
WITH ordered AS (
    SELECT income,
           ROW_NUMBER() OVER (ORDER BY income) AS rn,
           COUNT(*) OVER () AS total
    FROM creditcard_dedup
),
quartiles AS (
    SELECT
        MAX(CASE WHEN rn = CAST(total*0.25 AS INT) THEN income END) AS Q1,
        MAX(CASE WHEN rn = CAST(total*0.75 AS INT) THEN income END) AS Q3
    FROM ordered
)
SELECT *
FROM creditcard_dedup, quartiles
WHERE income BETWEEN (Q1 - 1.5*(Q3 - Q1)) AND (Q3 + 1.5*(Q3 - Q1)) LIMIT 10;




 * sqlite:////content/project.db
Done.


rownames,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active,rn,Q1,Q3
462,no,0,0.0,3.0,0.0004,0.0,no,no,0,18,0,2,1,2.236,4.0
1190,no,0,19.0,1.56,0.0007692308,0.0,yes,no,0,46,0,3,1,2.236,4.0
1151,no,0,19.0,1.92,0.000625,0.0,no,no,0,6,0,4,1,2.236,4.0
545,no,0,19.0,1.97,0.0006091371,0.0,no,no,1,24,1,2,1,2.236,4.0
965,no,0,19.0,2.1,0.0005714286,0.0,no,no,0,124,1,2,1,2.236,4.0
1269,no,0,19.0,2.2,0.0005454546,0.0,no,no,0,6,1,0,1,2.236,4.0
1207,no,0,20.0,1.2,0.001,0.0,no,yes,1,24,1,0,1,2.236,4.0
1203,no,0,20.0,1.8,0.0006666667,0.0,no,no,0,4,0,1,1,2.236,4.0
297,no,0,20.0,1.95,0.0006153846,0.0,no,no,0,14,0,1,1,2.236,4.0
136,no,0,20.0,2.64,0.0004545455,0.0,no,no,3,7,1,1,1,2.236,4.0


In [27]:
%%sql
-- Order by expenditure (highest first)
SELECT card, income, expenditure
FROM creditcard_dedup
ORDER BY expenditure DESC
LIMIT 10;



 * sqlite:////content/project.db
Done.


card,income,expenditure
yes,12.4999,3099.505
yes,6.0,2291.174
yes,10.4,2001.547
yes,9.0,1949.862
yes,2.6,1902.0
yes,4.8,1898.033
yes,3.7,1836.976
yes,2.6,1786.277
yes,5.0,1774.356
yes,3.65,1637.527


In [28]:
%%sql
-- Rank customers by expenditure (highest spender first)
SELECT
    card, income, expenditure,
    ROW_NUMBER() OVER (ORDER BY expenditure DESC) AS row_num,
    RANK() OVER (ORDER BY expenditure DESC) AS rank_pos
FROM creditcard_dedup
LIMIT 10

 * sqlite:////content/project.db
Done.


card,income,expenditure,row_num,rank_pos
yes,12.4999,3099.505,1,1
yes,6.0,2291.174,2,2
yes,10.4,2001.547,3,3
yes,9.0,1949.862,4,4
yes,2.6,1902.0,5,5
yes,4.8,1898.033,6,6
yes,3.7,1836.976,7,7
yes,2.6,1786.277,8,8
yes,5.0,1774.356,9,9
yes,3.65,1637.527,10,10
